In [1]:
import pandas as pd
import numpy as np
import re
from Bio.Seq import Seq

# Coordinates

In [2]:
emihu_gff = './Emihu1_all_genes.gff'

In [3]:
columns = ['scaffold', 'jgi', 'type', 'start', 'end', 'direction1', 'direction2', 'phase', 'infos']

# Load the GFF file
df_gff = pd.read_csv(emihu_gff, sep='\t', comment='#', names=columns)

# Split the 'attributes' column by ';' into multiple columns
df_gff_split = df_gff['infos'].str.split(';', expand=True)
# Rename the new columns (optional)
df_gff_split.columns = ['name', 'proteinId', 'n_exon']
df_gff.columns = df_gff.columns.str.strip()

# Add the split columns back to the original DataFrame
df_gff = pd.concat([df_gff, df_gff_split], axis=1)
df_gff.drop(['infos'], axis=1, inplace=True)

df_gff = df_gff[~df_gff['proteinId'].isna()]
# convert CDS position to integer and subtract by 1 because string start from 0 in python
df_gff['start'] = df_gff['start'].astype(int) - 1

In [4]:
df_gff = df_gff[~df_gff['proteinId'].isna()]

In [5]:
df_gff

,scaffold,jgi,type,start,end,direction1,direction2,phase,name,proteinId,n_exon
0,scaffold_1,JGI,exon,203,287,.,-,.,"name ""gm1.100001""",transcriptId 193701,None
1,scaffold_1,JGI,CDS,203,287,.,-,0,"name ""gm1.100001""",proteinId 193701,exonNumber 3
3,scaffold_1,JGI,exon,1701,1737,.,-,.,"name ""gm1.100001""",transcriptId 193701,None
4,scaffold_1,JGI,CDS,1701,1737,.,-,0,"name ""gm1.100001""",proteinId 193701,exonNumber 2
5,scaffold_1,JGI,exon,1939,2059,.,-,.,"name ""gm1.100001""",transcriptId 193701,None
...,...,...,...,...,...,...,...,...,...,...,...
2076640,scaffold_999,JGI,CDS,12061,12208,.,+,0,"name ""fgenesh_newKGs_pm.999__2""",proteinId 317413,exonNumber 1
2076642,scaffold_999,JGI,exon,12294,12343,.,+,.,"name ""fgenesh_newKGs_pm.999__2""",transcriptId 317413,None
2076643,scaffold_999,JGI,CDS,12294,12343,.,+,0,"name ""fgenesh_newKGs_pm.999__2""",proteinId 317413,exonNumber 2
2076644,scaffold_999,JGI,exon,12548,12760,.,+,.,"name ""fgenesh_newKGs_pm.999__2""",transcriptId 317413,None


## check integrity

In [6]:
df_gff['scaffold'].value_counts()

scaffold
scaffold_1       32226
scaffold_5       28201
scaffold_3       26272
scaffold_4       23537
scaffold_7       16370
                 ...  
scaffold_6462        2
scaffold_4460        2
scaffold_7819        2
scaffold_9088        2
scaffold_9683        2
Name: count, Length: 7760, dtype: int64

In [7]:
df_gff[df_gff['scaffold'] == 'scaffold_5']

,scaffold,jgi,type,start,end,direction1,direction2,phase,name,proteinId,n_exon
413483,scaffold_5,JGI,exon,243,654,.,-,.,"name ""gm1.500001""",transcriptId 196822,None
413484,scaffold_5,JGI,CDS,243,654,.,-,0,"name ""gm1.500001""",proteinId 196822,exonNumber 1
413487,scaffold_5,JGI,exon,739,763,.,+,.,"name ""gm1.500002""",transcriptId 196823,None
413488,scaffold_5,JGI,CDS,739,763,.,+,0,"name ""gm1.500002""",proteinId 196823,exonNumber 1
413490,scaffold_5,JGI,exon,1105,1204,.,+,.,"name ""gm1.500002""",transcriptId 196823,None
...,...,...,...,...,...,...,...,...,...,...,...
2057058,scaffold_5,JGI,CDS,2128589,2128617,.,-,0,"name ""fgenesh_newKGs_pm.5__155""",proteinId 308753,exonNumber 3
2057059,scaffold_5,JGI,exon,2128705,2128815,.,-,.,"name ""fgenesh_newKGs_pm.5__155""",transcriptId 308753,None
2057060,scaffold_5,JGI,CDS,2128705,2128815,.,-,1,"name ""fgenesh_newKGs_pm.5__155""",proteinId 308753,exonNumber 2
2057061,scaffold_5,JGI,exon,2128887,2128971,.,-,.,"name ""fgenesh_newKGs_pm.5__155""",transcriptId 308753,None


In [8]:
# get the first cds of protein which is translated by 5-3
result_53 = df_gff.groupby('name').apply(
    lambda group: group.loc[
        (group['type'] == 'CDS') & (group['direction2'] == '+')
    ].head(1)  # get first match
).reset_index(drop=True)

# get the last cds of protein which is translated by 3-5
result_35 = df_gff.groupby('name').apply(
    lambda group: group.loc[
        (group['type'] == 'CDS') & (group['direction2'] == '-')
    ].iloc[-1] if any((group['type'] == 'CDS') & (group['direction2'] == '-')) else None
).dropna().reset_index(drop=True)

/var/folders/2v/0h890r5d3g93n2j6tb6k9cf00000gn/T/ipykernel_87031/2718034935.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_53 = df_gff.groupby('name').apply(
/var/folders/2v/0h890r5d3g93n2j6tb6k9cf00000gn/T/ipykernel_87031/2718034935.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_35 = df_gff.groupby('name').apply(


In [9]:
result = pd.concat((result_53, result_35))
result

,scaffold,jgi,type,start,end,direction1,direction2,phase,name,proteinId,n_exon
0,scaffold_1,JGI,CDS,1273784.0,1273873.0,.,+,0,"name ""e_gw1.1.100.1""",proteinId 60925,exonNumber 1
1,scaffold_1,JGI,CDS,1238222.0,1238786.0,.,+,0,"name ""e_gw1.1.102.1""",proteinId 61047,exonNumber 1
2,scaffold_1,JGI,CDS,1999638.0,2000029.0,.,+,0,"name ""e_gw1.1.103.1""",proteinId 61109,exonNumber 1
3,scaffold_1,JGI,CDS,857135.0,857702.0,.,+,0,"name ""e_gw1.1.105.1""",proteinId 60920,exonNumber 1
4,scaffold_1,JGI,CDS,1189068.0,1189311.0,.,+,0,"name ""e_gw1.1.106.1""",proteinId 60980,exonNumber 1
...,...,...,...,...,...,...,...,...,...,...,...
113277,scaffold_998,JGI,CDS,5716.0,5846.0,.,-,2,"name ""gw1.998.2.1""",proteinId 46252,exonNumber 1
113278,scaffold_998,JGI,CDS,5716.0,5858.0,.,-,2,"name ""gw1.998.3.1""",proteinId 46444,exonNumber 1
113279,scaffold_998,JGI,CDS,67.0,379.0,.,-,0,"name ""gw1.998.4.1""",proteinId 59951,exonNumber 1
113280,scaffold_998,JGI,CDS,67.0,379.0,.,-,0,"name ""gw1.998.5.1""",proteinId 59974,exonNumber 1


In [10]:
result.loc[result['direction2'] == '-']['scaffold'].value_counts()

scaffold
scaffold_1       2096
scaffold_5       1642
scaffold_4       1591
scaffold_3       1579
scaffold_7       1031
                 ... 
scaffold_3133       1
scaffold_3129       1
scaffold_3124       1
scaffold_3123       1
scaffold_9996       1
Name: count, Length: 5247, dtype: int64

# Fasta Sequence

In [11]:
emihu_seq_path = './Emihu1_masked_scaffolds.fasta'
emihu_seq_dict = {}
key = ''
with open(emihu_seq_path, 'r') as f:
    for line in f.readlines():
        if ">" in line:
            # save scaffold number as dict key
            key = line[1:].strip()  # avoid '>'
            # create a value for storing the sequence
            emihu_seq_dict[key] = ''
        else:
            emihu_seq_dict[key] += line.strip()


## verify

In [12]:
result[result['name'] == 'name "gw1.13.147.1"']

,scaffold,jgi,type,start,end,direction1,direction2,phase,name,proteinId,n_exon
107078,scaffold_13,JGI,CDS,227696.0,227915.0,.,+,0,"name ""gw1.13.147.1""",proteinId 59809,exonNumber 1


In [13]:
tmp = emihu_seq_dict['scaffold_13'][
    int(result[result['name'] == 'name "gw1.13.147.1"']['start']):int(result[result['name'] == 'name "gw1.13.147.1"']['end'])
]

/var/folders/2v/0h890r5d3g93n2j6tb6k9cf00000gn/T/ipykernel_87031/2129101991.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  int(result[result['name'] == 'name "gw1.13.147.1"']['start']):int(result[result['name'] == 'name "gw1.13.147.1"']['end'])


In [14]:
len(tmp), tmp

(219,
 'GGTCCCGTCATCGGCATCGACCTCGGCACCACCTATTCCTGCGTCGGAGTCTACCGGAAGGGCAAGGTGGAGATCATCGCCAACGAGCAGGGGAACCGAGTGACGCCGTCGTGGGTGGCGTTCACCGACGACGGCGAGCGGCTGGTCGGCGACGCCGCGCGCTCGCAGTCCTCGCTCAACCCGGTAAACACGATTTACGACGCGAAGCGGCTCATCGGC')

<font color='red'> Same result shown on JGI webpage

### Find the closest M to * in the inverse direction from CDS to *   

In [15]:

# Example DNA sequence
dna_seq = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")

# Translate to protein
protein_seq = dna_seq.translate()

print(protein_seq)

MAIVMGR*KGAR*


In [16]:
def reVerifyM(
        fasta_seq: str, 
        cds_start: float,
        cds_end:float,
        direction:str,
        ) -> int:
    """
    Take an input of fasta acide amine sequence, CDS position from JGI 
    and re-find the M position in the given sequence
    """
    cds_start = int(cds_start)
    if cds_start < 0 or cds_start >= len(fasta_seq) or cds_start > cds_end:
        return cds_start

    cds_end = int(cds_end)
    if cds_end < 0 or cds_end >= len(fasta_seq):
        return cds_start
    # Forward
    if direction == "+":
        # translate DNA to AA
        # include the first translated aa
        sub_seq = fasta_seq[:cds_start+3]
        # identify the translation framework: +0,+1 or +2 depend on the length of sequence
        framework = len(sub_seq)%3
        sub_seq = str(Seq(sub_seq[framework:]).translate())
        # find the closest codon stop
        # consider the closest * near the cds start position so inverse make the search run faster
        inversed_seq = sub_seq[::-1]
        match = re.search(r'\*', inversed_seq)
        # the first codon stop encoutered
        stop_pos = match.start() if match else len(sub_seq)

        # find the closest aa M to the first * by slicing again the inversed aa sequence
        tmp_seq = inversed_seq[:stop_pos]
        # find the first M encoutered
        M_pos = tmp_seq.find('M')
        if M_pos >= 0:
            M_pos = (len(inversed_seq) - M_pos - 1)*3 + framework
        return M_pos
    # Backward
    else:
        # find the true (furthest) M position by going backward the CDS sequence
        sub_seq = fasta_seq[cds_start-3: cds_end]
        sub_seq = str(Seq(sub_seq).reverse_complement().translate())
        # print(sub_seq)
        # find the last M encoutered
        inversed_seq = sub_seq[::-1]
        M_pos = inversed_seq.find('M')
        if M_pos >= 0:
            M_pos = len(inversed_seq) - M_pos - 1
            # print(sub_seq[M_pos:])
            # print(cds_end - M_pos*3)
            # print(Seq(fasta_seq[cds_start:cds_end-M_pos*3]).reverse_complement().translate())
            return cds_end - M_pos*3
        return int(M_pos)

In [17]:
tmp

'GGTCCCGTCATCGGCATCGACCTCGGCACCACCTATTCCTGCGTCGGAGTCTACCGGAAGGGCAAGGTGGAGATCATCGCCAACGAGCAGGGGAACCGAGTGACGCCGTCGTGGGTGGCGTTCACCGACGACGGCGAGCGGCTGGTCGGCGACGCCGCGCGCTCGCAGTCCTCGCTCAACCCGGTAAACACGATTTACGACGCGAAGCGGCTCATCGGC'

In [18]:
Seq(tmp).translate()

Seq('GPVIGIDLGTTYSCVGVYRKGKVEIIANEQGNRVTPSWVAFTDDGERLVGDAAR...LIG')

In [19]:
len(emihu_seq_dict[result[result['name'] == 'name "gw1.13.147.1"']['scaffold'].iloc[0]][:int(result[result['name'] == 'name "gw1.13.147.1"']['start'])])

/var/folders/2v/0h890r5d3g93n2j6tb6k9cf00000gn/T/ipykernel_87031/2200201372.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  len(emihu_seq_dict[result[result['name'] == 'name "gw1.13.147.1"']['scaffold'].iloc[0]][:int(result[result['name'] == 'name "gw1.13.147.1"']['start'])])


227696

In [20]:
227696 // 3, 227696 % 3,  

(75898, 2)

In [21]:
# cds_pos = int(result.loc[result['name'] == 'name "gw1.13.147.1"', 'start'].iloc[0])
# cds_end = int(result.loc[result['name'] == 'name "gw1.13.147.1"', 'end'].iloc[0])
# scaffold = result.loc[result['name'] == 'name "gw1.13.147.1"', 'scaffold'].iloc[0]
# print(Seq(emihu_seq_dict[scaffold][cds_pos:cds_end]).translate())
# m_pos = reVerifyM(emihu_seq_dict[scaffold],
#       cds_pos=cds_pos
# )
# m_pos

In [23]:
# Seq(emihu_seq_dict[result[result['name'] == 'name "gw1.13.147.1"']['scaffold'].iloc[0]][
#     m_pos:int(result.loc[result['name'] == 'name "gw1.13.147.1"', 'end'].iloc[0])
# ]).translate()

In [24]:
# result = result[~result['proteinId'].isna()]

In [25]:
# result[result['proteinId'].str.contains('421892')]

In [50]:
# cds_pos = int(result.loc[result['name'] == 'name "estExtDG_fgenesh_newKGs_pm.C_470006"', 'start'].iloc[0])
# m_pos = reVerifyM(emihu_seq_dict[result.loc[result['name'] == 'name "estExtDG_fgenesh_newKGs_pm.C_470006"', 'scaffold'].iloc[0]],
#       cds_pos=cds_pos
# )
# print(m_pos)
# print(Seq(emihu_seq_dict[result.loc[result['name'] == 'name "estExtDG_fgenesh_newKGs_pm.C_470006"', 'scaffold'].iloc[0]][
#     int(result.loc[result['name'] == 'name "estExtDG_fgenesh_newKGs_pm.C_470006"', 'start'].iloc[0]):int(result.loc[result['name'] == 'name "estExtDG_fgenesh_newKGs_pm.C_470006"', 'end'].iloc[0])
# ]).translate())
# Seq(emihu_seq_dict[result.loc[result['name'] == 'name "estExtDG_fgenesh_newKGs_pm.C_470006"', 'scaffold'].iloc[0]][
#     m_pos:int(result.loc[result['name'] == 'name "estExtDG_fgenesh_newKGs_pm.C_470006"', 'end'].iloc[0])
# ]).translate()

166785
MRLLITLIPLLAAAATAEEAEAVGPVIGIDLGTTYSCVGVYRKGKVEIIANEQGNRVTPSWVAFTDDGERLVGDAARSQSSLNPVNTIYDAKRLIGRGFRDEEVQKDAEHWPFKVVATAEGKPAVEVAAGGATKRLLPQEISAMVLGKMKG


Seq('MRLLITLIPLLAAAATAEEAEAVGPVIGIDLGTTYSCVGVYRKGKVEIIANEQG...MKG')

In [26]:
results = []

for scaffold, group_df in result.groupby(['scaffold']):
    fasta_seq = emihu_seq_dict[scaffold[0]]  # get once
    starts = group_df['start'].astype(int).tolist()
    ends = group_df['end'].astype(int).tolist()
    directions = group_df['direction2'].tolist()

    # Vectorized by list comprehension per group
    true_M_positions = [reVerifyM(fasta_seq, cds_start, cds_end, direction) for (cds_start, cds_end, direction) in zip(starts, ends, directions)]

    group_result = group_df.copy()
    group_result['true_M_pos'] = true_M_positions
    group_result['no_M'] = False
    group_result.loc[group_result['true_M_pos'] == -1, 'no_M'] = True 

    # calculate the differences
    group_result['M_pos_diff'] = 0
    # only gene in which has M can calculate the difference
    mask = ~group_result['no_M']
    
    # forward because of direction +
    mask2 = group_result['direction2'] == '+'
    group_result.loc[mask & mask2, 'M_pos_diff'] = (
        group_result.loc[mask & mask2, 'start'] - group_result.loc[mask & mask2, 'true_M_pos']
    ).abs()

    # backward because of direction -
    mask3 = group_result['direction2'] == '-'
    group_result.loc[mask & mask3, 'M_pos_diff'] = (
        group_result.loc[mask & mask3, 'end'] - group_result.loc[mask & mask3, 'true_M_pos']
    ).abs()
    results.append(group_result)
    break 

# Concatenate all groups
result = pd.concat(results, ignore_index=True)

/Users/tranquocan.truong/Documents/AnTTQ/U_Sorbonne/M1S2/Projet/projectS1/lib/python3.9/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [27]:
result

,scaffold,jgi,type,start,end,direction1,direction2,phase,name,proteinId,n_exon,true_M_pos,no_M,M_pos_diff
0,scaffold_1,JGI,CDS,1273784.0,1273873.0,.,+,0,"name ""e_gw1.1.100.1""",proteinId 60925,exonNumber 1,-1,True,0
1,scaffold_1,JGI,CDS,1238222.0,1238786.0,.,+,0,"name ""e_gw1.1.102.1""",proteinId 61047,exonNumber 1,-1,True,0
2,scaffold_1,JGI,CDS,1999638.0,2000029.0,.,+,0,"name ""e_gw1.1.103.1""",proteinId 61109,exonNumber 1,1999638,False,0
3,scaffold_1,JGI,CDS,857135.0,857702.0,.,+,0,"name ""e_gw1.1.105.1""",proteinId 60920,exonNumber 1,857135,False,0
4,scaffold_1,JGI,CDS,1189068.0,1189311.0,.,+,0,"name ""e_gw1.1.106.1""",proteinId 60980,exonNumber 1,-1,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4190,scaffold_1,JGI,CDS,1456659.0,1456868.0,.,-,1,"name ""gw1.1.87.1""",proteinId 45700,exonNumber 1,1456784,False,84
4191,scaffold_1,JGI,CDS,2843908.0,2844120.0,.,-,1,"name ""gw1.1.89.1""",proteinId 45750,exonNumber 1,2844069,False,51
4192,scaffold_1,JGI,CDS,2705691.0,2705999.0,.,-,1,"name ""gw1.1.90.1""",proteinId 45766,exonNumber 1,2705984,False,15
4193,scaffold_1,JGI,CDS,1629617.0,1629799.0,.,-,1,"name ""gw1.1.92.1""",proteinId 46190,exonNumber 1,1629625,False,174


In [28]:
result[result['direction2'] == '+']

,scaffold,jgi,type,start,end,direction1,direction2,phase,name,proteinId,n_exon,true_M_pos,no_M,M_pos_diff
0,scaffold_1,JGI,CDS,1273784.0,1273873.0,.,+,0,"name ""e_gw1.1.100.1""",proteinId 60925,exonNumber 1,-1,True,0
1,scaffold_1,JGI,CDS,1238222.0,1238786.0,.,+,0,"name ""e_gw1.1.102.1""",proteinId 61047,exonNumber 1,-1,True,0
2,scaffold_1,JGI,CDS,1999638.0,2000029.0,.,+,0,"name ""e_gw1.1.103.1""",proteinId 61109,exonNumber 1,1999638,False,0
3,scaffold_1,JGI,CDS,857135.0,857702.0,.,+,0,"name ""e_gw1.1.105.1""",proteinId 60920,exonNumber 1,857135,False,0
4,scaffold_1,JGI,CDS,1189068.0,1189311.0,.,+,0,"name ""e_gw1.1.106.1""",proteinId 60980,exonNumber 1,-1,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094,scaffold_1,JGI,CDS,1394773.0,1395190.0,.,+,0,"name ""gw1.1.94.1""",proteinId 46209,exonNumber 1,1394773,False,0
2095,scaffold_1,JGI,CDS,2885127.0,2885541.0,.,+,0,"name ""gw1.1.95.1""",proteinId 46224,exonNumber 1,-1,True,0
2096,scaffold_1,JGI,CDS,1898490.0,1898835.0,.,+,0,"name ""gw1.1.96.1""",proteinId 46246,exonNumber 1,1898307,False,183
2097,scaffold_1,JGI,CDS,718868.0,719048.0,.,+,0,"name ""gw1.1.97.1""",proteinId 46269,exonNumber 1,-1,True,0


In [30]:

print(Seq(emihu_seq_dict['scaffold_1'][
    int(1999639.0-1):int(2000029.0)
]).translate())


MDKTLKIRGNPKVFFDMEIGGEPAGRVVMQLRADVVPKTAENFRQLCLKEEGEGYKGSSFHRVIPGFMCQGGDFTNHNGTGGKSIYGEKFEDENFQLKHTGKGVLSMANAGPGTNGSQFFLCTAKTAWLD


In [31]:
result[result['direction2'] == '-']

,scaffold,jgi,type,start,end,direction1,direction2,phase,name,proteinId,n_exon,true_M_pos,no_M,M_pos_diff
2099,scaffold_1,JGI,CDS,1234941.0,1235519.0,.,-,1,"name ""e_gw1.1.1.1""",proteinId 61078,exonNumber 1,1235144,False,375
2100,scaffold_1,JGI,CDS,548869.0,549137.0,.,-,2,"name ""e_gw1.1.104.1""",proteinId 61169,exonNumber 1,548957,False,180
2101,scaffold_1,JGI,CDS,548869.0,549137.0,.,-,2,"name ""e_gw1.1.107.1""",proteinId 61038,exonNumber 1,548957,False,180
2102,scaffold_1,JGI,CDS,548869.0,549137.0,.,-,2,"name ""e_gw1.1.108.1""",proteinId 61099,exonNumber 1,548957,False,180
2103,scaffold_1,JGI,CDS,2686721.0,2687227.0,.,-,1,"name ""e_gw1.1.110.1""",proteinId 61045,exonNumber 1,2687227,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4190,scaffold_1,JGI,CDS,1456659.0,1456868.0,.,-,1,"name ""gw1.1.87.1""",proteinId 45700,exonNumber 1,1456784,False,84
4191,scaffold_1,JGI,CDS,2843908.0,2844120.0,.,-,1,"name ""gw1.1.89.1""",proteinId 45750,exonNumber 1,2844069,False,51
4192,scaffold_1,JGI,CDS,2705691.0,2705999.0,.,-,1,"name ""gw1.1.90.1""",proteinId 45766,exonNumber 1,2705984,False,15
4193,scaffold_1,JGI,CDS,1629617.0,1629799.0,.,-,1,"name ""gw1.1.92.1""",proteinId 46190,exonNumber 1,1629625,False,174


In [34]:
print(Seq(emihu_seq_dict['scaffold_1'][
    int(548869.0-3):int(548957.0)
]).reverse_complement().translate())

MSLGGEGRGQFDSAIDAAYDAGVLTVVAAG


In [199]:
cds_start = 548869.0
cds_end = 549137.0
m_pos = reVerifyM(emihu_seq_dict['scaffold_1'],
      cds_start,
      cds_end,
      '-'
)
print(m_pos)

MSLGGEGRGQFDSAIDAAYDAGVLTVVAAG
548957
MSLGGEGRGQFDSAIDAAYDAGVLTVVAA
548956


/Users/tranquocan.truong/Documents/AnTTQ/U_Sorbonne/M1S2/Projet/projectS1/lib/python3.9/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [203]:
print(Seq(emihu_seq_dict['scaffold_1'][
    int(548869.0-3):int(548957)
]).reverse_complement().translate())

MSLGGEGRGQFDSAIDAAYDAGVLTVVAAG


/Users/tranquocan.truong/Documents/AnTTQ/U_Sorbonne/M1S2/Projet/projectS1/lib/python3.9/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [104]:
# cds_pos = int(result.loc[result['name'] == 'name "gw1.1.87.1"', 'start'].iloc[0])
# m_pos = reVerifyM(emihu_seq_dict[result.loc[result['name'] == 'name "gw1.1.87.1"', 'scaffold'].iloc[0]],
#       cds_pos=cds_pos,
#       direction=result.loc[result['name'] == 'name "gw1.1.87.1"', 'direction2'].iloc[0]
# )
# print(cds_pos)
# print(m_pos)

print(Seq(emihu_seq_dict['scaffold_1'][
    int(1456659.0):int(1456868.0)
]).reverse_complement().translate())

# Seq(emihu_seq_dict[result.loc[result['name'] == 'name "e_gw1.1.1.1"', 'scaffold'].iloc[0]][
#     m_pos:int(result.loc[result['name'] == 'name "e_gw1.1.1.1"', 'end'].iloc[0])
# ]).translate()

GLLSNYEYLCHLNDAAGRSCADLAQYPVMPWVLQDYTSHTLDLADPAVYRDLSKPVGALDASRLALFRE


In [ ]:
# testing e_gw1.1.100.1: why does it not start with M but is_diff = 0
M_pos = 0
stop_flag = False
tmp_seq =  str(Seq(emihu_seq_dict['scaffold_1'][:1273784]).translate())[::-1]
print(str(Seq(emihu_seq_dict['scaffold_1'][1273784:1273873]).translate()))
for c in range(len(tmp_seq)):
    print(tmp_seq[c])
    if tmp_seq[c] == "M" and not stop_flag:
        M_pos = c
    if tmp_seq[c] == '*':
        break

M_pos

In [ ]:
cds_pos = int(result[result['name'] == 'name "e_gw1.1.100.1"']['start'])
m_pos = reVerifyM(emihu_seq_dict[result[result['name'] == 'name "e_gw1.1.100.1"']['scaffold'].iloc[0]],
      cds_pos=cds_pos
)
Seq(emihu_seq_dict[result[result['name'] == 'name "e_gw1.1.100.1"']['scaffold'].iloc[0]][
    m_pos:int(result[result['name'] == 'name "e_gw1.1.100.1"']['end'])
]).translate()

In [ ]:
result[result['is_diff'] != 0]

In [ ]:
43566 / 228000

# packaging

In [31]:
import os 


def load_gff(file_path:str ='./Emihu1_all_genes.gff', 
             columns:list=['scaffold', 'jgi', 'type', 'start', 'end', 'direction1', 'direction2', 'phase', 'infos']
             ) -> pd.DataFrame:
    """
    Read table from .GFF file and load in to Pandas.DataFrame table
    """
    # check path
    if not os.path.exists(file_path):
        print("Non existed input path, Please re-check!")
        return None
    
    # Load the GFF file
    df_gff = pd.read_csv(file_path, sep='\t', comment='#', names=columns)

    # Split the 'attributes' column by ';' into multiple columns
    df_gff_split = df_gff['infos'].str.split(';', expand=True)
    # Rename the new columns (optional)
    df_gff_split.columns = ['name', 'proteinId', 'n_exon']
    df_gff.columns = df_gff.columns.str.strip()

    # Add the split columns back to the original DataFrame
    df_gff = pd.concat([df_gff, df_gff_split], axis=1)
    df_gff.drop(['infos'], axis=1, inplace=True)
    df_gff = df_gff[~df_gff['proteinId'].isna()]
    # convert CDS position to integer and subtract by 1 because string start from 0 in python
    df_gff['start'] = df_gff['start'].astype(int) - 1
    
    return df_gff


def get_Coordinates(
        df_gff:pd.DataFrame, 
        groupby_attr:str ='name'
        ) -> pd.DataFrame:
    """
    Filtering the first CDS coordinate infomation for each attribute ('Name' by default) group
    from the GFF dataframe 
    """

    # if not df_gff.isinstance(pd.DataFrame):
    #     print("Non existed input path, Please re-check!")
    #     return None
    
    # Group by 'group' and find the first row with a non-null value
    result = df_gff.groupby(groupby_attr, group_keys=False).apply(
        lambda group: group.loc[group[group['type'] == 'CDS'].index[0]] if any(group['type'] == 'CDS') else None,
        include_groups=False
    )

    return result

def load_Fasta(fasta_path:str = './Emihu1_masked_scaffolds.fasta'):
    
    if not os.path.exists(fasta_path):
        print("Non existed input path, Please re-check!")
        return None
    
    ret_dict = {}
    key = ''
    with open(fasta_path, 'r') as f:
        for line in f.readlines():
            if ">" in line:
                # translate acide amine sequence to protein before advance
                if key != '':
                    ret_dict[key] = ret_dict[key]
                # save scaffold number as dict key
                key = line[1:].strip()  # avoid '>'
                # create a value for storing the sequence
                ret_dict[key] = ''
            else:
                ret_dict[key] += line.strip()
    f.close()

    return ret_dict


def reVerifyM(
        fasta_seq: str, 
        cds_pos: float
        ) -> int:
    """
    Take an input of fasta acide amine sequence, CDS position from JGI 
    and re-find the M position in the given sequence
    """
    cds_pos = int(cds_pos)
    if cds_pos < 0 or cds_pos >= len(fasta_seq):
        return cds_pos

    # include the first translated codon
    sub_seq = fasta_seq[:cds_pos+3]
    # print(len(sub_seq))
    # identify the translation framework: +0,+1 or +2 depend on the length of sequence
    framework = len(sub_seq)%3
    sub_seq = str(Seq(sub_seq[framework:]).translate())
    # print(len(sub_seq))
    
    # find the closest codon stop
    star_indices = [m.start() for m in re.finditer(r'\*', sub_seq)]
    if not star_indices:
        return cds_pos
    closest_stop = min(star_indices, key=lambda i: abs(i - cds_pos))
    # print(closest_stop)
    stop_pos = closest_stop*3 + framework
    # print(stop_pos)

    # find the closest aa M to the closest codon stop *
    sub_seq = sub_seq[closest_stop:]
    star_indices = [m.start() for m in re.finditer(r'M', sub_seq)]
    if not star_indices:
        return -1 # mean there is no M from the position cds+1 to the closest *
    closest_M = min(star_indices, key=lambda i: abs(i - closest_stop))
    M_pos = closest_M*3 + stop_pos
    # print(M_pos)
    # print(Seq(fasta_seq[M_pos:cds_pos+3]).translate())
    # sub_seq = sub_seq[closest_M:]

    # print(sub_seq)
    # print(closest_stop)

    return int(M_pos)

!! Why not translate DNA to protein before saving into dict?
=> To respect the framework of translation (+1, +2, +3). If we translate before saving, we can translate the wrong framework compare to the true translated one then we'll translate after getting the sub-sequence before the CDS Position. 

In [32]:
df_gff_emihu1 = load_gff(emihu_gff)
fasta_dict_emihu1 = load_Fasta(emihu_seq_path)
df_cds_emihu1 = get_Coordinates(df_gff_emihu1)

In [ ]:
df_gff_emihu1

In [ ]:
df_cds_emihu1

In [ ]:
type(df_cds_emihu1)

In [ ]:
df_cds_emihu1[df_cds_emihu1.index == 'name "gw1.13.147.1"']

In [ ]:
tmp = fasta_dict_emihu1['scaffold_13'][
    int(df_cds_emihu1[df_cds_emihu1.index == 'name "gw1.13.147.1"']['start']):int(df_cds_emihu1[df_cds_emihu1.index == 'name "gw1.13.147.1"']['end']+1)
]

In [ ]:
len(tmp), tmp

! ALL GOOD